# Windowing module

Imports

In [1]:
import numpy as np
from tqdm import tqdm

from data_access.insert.transit import insert_transit_windows
from data_access.read.flux import get_m_a_trend_flux_for_star_in_sector, get_usual_stars_for_sector
from tools.percentage import calculate_percentage


class WindowIsTooWideException(Exception):
    ...

### Variables Selection

In [5]:
sector = 2
minimum_half_transit_width_half = 0.3
maximum_half_transit_width_half = 2.0
start_half_transit_width_half = 3.1
half_transit_step = -0.05
trend_percentage = 0.95

### Process

In [6]:
tics = get_usual_stars_for_sector(sector)
for tic in tqdm(tics):
    df = get_m_a_trend_flux_for_star_in_sector(sector, tic)
    df.reset_index(inplace=True)
    df.rename(columns={'id': 'db_id'}, inplace=True)

    df['m_a_trend_flux_diff'] = df['m_a_trend_flux'].diff()
    df['m_a_trend_flux_diff_positive'] = df['m_a_trend_flux_diff'] > 0
    df['m_a_trend_flux_change_to_true'] = ((df['m_a_trend_flux_diff_positive'] == True) &
                                           (df['m_a_trend_flux_diff_positive'].shift(1) == False))

    df.loc[:,f'width_of_biggest_neg_trend_to_left'] = np.nan
    df.loc[:,f'width_of_biggest_pos_trend_to_right'] = np.nan

    df_filtered_with_changing_points = df[df['m_a_trend_flux_change_to_true'] == True]
    for i in df_filtered_with_changing_points.index:
        db_id = df.at[i, 'db_id']
        time = df.at[i, 'time']
        try:
            # Start from the wider windows and gradually narrow its width
            for width_before in np.arange(start_half_transit_width_half, minimum_half_transit_width_half, half_transit_step):
                # Find the starting index of the window
                before_start_idx = df['time'].searchsorted(df['time'].iloc[i] - width_before, side='right')
                before_start_idx = max(before_start_idx, 0)

                # Calculate the percentage of negative values before the point
                segment = df['m_a_trend_flux_diff_positive'].iloc[before_start_idx:i]
                negatives_before_percentage = calculate_percentage(segment, width_before, positive=False)
                if negatives_before_percentage > trend_percentage:
                    if width_before > maximum_half_transit_width_half:
                        # we don't need wider trends than maximum_half_transit_width_half
                        raise WindowIsTooWideException()

                    # Assign the value of the biggest negative trend found
                    df.at[i, 'width_of_biggest_neg_trend_to_left'] = width_before
                    df.at[i, 'transit_start_flux_id'] = df.at[before_start_idx, 'db_id']
                    df.at[i, 'transit_depth_left'] = df.at[before_start_idx, 'm_a_trend_flux'] - df.at[i, 'm_a_trend_flux']

                    # End the loop if the appropriate window is found
                    break

        except WindowIsTooWideException:
            continue

    df_filtered_with_negative_trends = df[df['width_of_biggest_neg_trend_to_left'].notna()]
    for i, row in df_filtered_with_negative_trends.iterrows():
        width_of_biggest_neg_trend_to_left = row['width_of_biggest_neg_trend_to_left']

        try:
            # right wing of a transit could be wider than the left one
            for width_after in np.arange(5 * width_of_biggest_neg_trend_to_left, width_of_biggest_neg_trend_to_left, half_transit_step):
                # Find the ending index of the window
                time_indices = df['time'].searchsorted(df['time'] + width_after, side='right') - 1
                after_end_idx = time_indices[i]
                after_end_idx = min(after_end_idx, len(df) - 1)

                # Calculate the percentage of positive values after the point
                segment = df['m_a_trend_flux_diff_positive'].iloc[i+1:after_end_idx+1]
                positives_after_percentage = calculate_percentage(segment, width_after, positive=True)
                if positives_after_percentage > trend_percentage:
                    if width_after > 4 * width_of_biggest_neg_trend_to_left:
                        # we don't need wider trends than 4*width_of_biggest_neg_trend_to_left
                        raise WindowIsTooWideException()
                    # Assign the value of the biggest positive trend found
                    df.at[i, 'width_of_biggest_pos_trend_to_right'] = width_after
                    df.at[i, 'transit_end_flux_id'] = df.at[after_end_idx, 'db_id']
                    df.at[i, 'transit_depth_right'] = df.at[after_end_idx, 'm_a_trend_flux'] - df.at[i, 'm_a_trend_flux']
                    # End the loop if the appropriate window is found
                    break

        except WindowIsTooWideException:
            continue

    df_filtered_with_both_trends = df[df['width_of_biggest_pos_trend_to_right'].notna()]

    insert_transit_windows(df_filtered_with_both_trends)

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:44<00:00,  1.70s/it]
